### Часть 1. Pix2Pix

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
%cd drive/MyDrive/HW3

In [3]:
from tqdm import tqdm
import numpy as np
import torch
from torchvision import datasets, transforms

from models import GeneratorUNet, Discriminator
from utils import show_results, init_normal_weights, set_requires_grad

In [5]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))])
dataset = datasets.ImageFolder('train_facades', transform=transform)
val_dataset = datasets.ImageFolder('val_facades', transform=transform)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=1)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=len(val_dataset))

In [6]:
nepochs = 150
mask_border = 256
lr_generator = 3e-4
lr_discriminator = 2e-4
alpha = 0.09
pixel_coeff = 50
discriminator_coeff = 0.5


generator = GeneratorUNet()
generator = generator.to(device)
generator.apply(init_normal_weights)
discriminator = Discriminator()
discriminator = discriminator.to(device)
discriminator.apply(init_normal_weights)


criterion_GAN = torch.nn.MSELoss().to(device)
criterion_pixelwise = torch.nn.L1Loss().to(device)
optimizer_G = torch.optim.Adam(generator.parameters(), lr=lr_generator, betas=(0.5, 0.999))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=lr_discriminator, betas=(0.5, 0.999))


for epoch in tqdm(range(nepochs)):
    for i, batch in enumerate(dataloader):
        real_A = batch[0][:, :, :, mask_border:].to(device)
        real_B = batch[0][:, :, :, :mask_border].to(device)
        valid = torch.tensor(np.ones((real_A.size(0), 1, 16, 16)), dtype=torch.float).to(device)
        fake = torch.tensor(np.zeros((real_A.size(0), 1, 16, 16)), dtype=torch.float).to(device)

        optimizer_G.zero_grad()
        fake_B = generator(real_A)
        pred_fake = discriminator(fake_B, real_A)

        # GAN loss
        loss_GAN = criterion_GAN(pred_fake, valid)

        # Pixel loss
        loss_pixel = criterion_pixelwise(fake_B, real_B)

        # Generator loss
        loss_G = alpha * loss_GAN + alpha * pixel_coeff * loss_pixel
        loss_G.backward()
        set_requires_grad(discriminator, False)
        optimizer_G.step()

        # Discriminator loss
        set_requires_grad(discriminator, True)
        optimizer_D.zero_grad()
        pred_real = discriminator(real_B, real_A)
        loss_real = criterion_GAN(pred_real, valid)
        pred_fake = discriminator(fake_B.detach(), real_A)
        loss_fake = criterion_GAN(pred_fake, fake)
        loss_D = discriminator_coeff * (loss_real + loss_fake)
        loss_D.backward()
        optimizer_D.step()

    print(f"Epoch {epoch} of {nepochs} | Loss_GAN: {round(loss_GAN.item(), 4)}, Loss_Pixel: {round(loss_pixel.item(), 4)}, Loss_Generator: {round(loss_G.item(), 4)} | Loss_Discriminator: {round(loss_D.item(), 4)}")

    if epoch % 5 == 0:
        imgs = next(iter(val_dataloader))
        ix = np.random.randint(0, len(val_dataset), 5).tolist()
        real_A = imgs[0][ix, :, :, mask_border:].to(device)
        real_B = imgs[0][ix, :, :, :mask_border]
        show_results(real_B, generator(real_A).to(torch.device("cpu")))

Output hidden; open in https://colab.research.google.com to view.